In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from argparse import Namespace
import yaml
import sys
import numpy as np  
from pathlib import Path
from rdkit import Chem
import random

REPO_ROOT = "/mnt/STORAGE3/sebastian2/DiffSBDD"
if REPO_ROOT not in sys.path:
    sys.path.append(REPO_ROOT)

from lightning_modules import LigandPocketDDPM

def to_ns(d):
    if isinstance(d, dict):
        return Namespace(**{k: to_ns(v) for k, v in d.items()})
    return d

def load_model_using_config(config, ckpt_path=None):
    with open(config, "r") as f:
        cfg = yaml.safe_load(f)
    args = to_ns(cfg)

    # args.wandb_params.mode = "disabled"  # disable wandb in notebook runs
    args.enable_progress_bar = True


    # Required histogram file from the processed dataset
    histogram_file = Path(args.datadir, "size_distribution.npy")
    histogram = np.load(histogram_file).tolist()

    # Build LightningModule with text conditioning
    pl_module = LigandPocketDDPM(
        outdir=Path(args.logdir, args.run_name),
        dataset=args.dataset,
        datadir=args.datadir,
        batch_size=args.batch_size,
        lr=args.lr,
        egnn_params=args.egnn_params,
        diffusion_params=args.diffusion_params,
        num_workers=args.num_workers,
        augment_noise=args.augment_noise,
        augment_rotation=args.augment_rotation,
        clip_grad=args.clip_grad,
        eval_epochs=args.eval_epochs,
        eval_params=args.eval_params,
        visualize_sample_epoch=args.visualize_sample_epoch,
        visualize_chain_epoch=args.visualize_chain_epoch,
        auxiliary_loss=args.auxiliary_loss,
        loss_params=args.loss_params,
        mode=args.mode,
        node_histogram=histogram,
        pocket_representation=args.pocket_representation,
        text_model_name=args.text_model_name,
        text_embeddings_path=args.text_embeddings_path,
        # text_csv=TEXT_CSV,
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if ckpt_path is not None:
        checkpoint = torch.load(ckpt_path, map_location='cpu')
        missing_values, unmatched_values = pl_module.load_state_dict(checkpoint['state_dict'], strict=False)
        print(f"Missing values: {missing_values}")
        print(f"Unmatched values: {unmatched_values}")
    pl_module = pl_module.to(device)
    pl_module.eval()
    
    return pl_module
    

In [7]:


# ---- User inputs ----
CKPT_PATH = "/mnt/STORAGE3/sebastian2/data_hackathon/logs/all_text_embeddings_llama/checkpoints/best-model-epoch=epoch=42.ckpt"
CONFIG_YML = "/mnt/STORAGE3/sebastian2/DiffSBDD/configs/FT_crossdock_fullatom_cond_llama.yml"
TEXT_DESCRIPTION = """
Generate a molecule with moderate molecular weight, low logP, high topological polar surface area, moderate hydrogen bond donors, high hydrogen bond acceptors, limited rotatable bonds, several rings, majority aromatic rings, balanced sp3 hybridization, and a neutral formal charge."""

N_SAMPLES = 20
POCKET_PDB = "/mnt/STORAGE3/sebastian2/DiffSBDD/example/5ndu.pdb"  # Example PDB file#
LIGAND_SDF = "/mnt/STORAGE3/sebastian2/DiffSBDD/example/5ndu_C_8V2.sdf"  # Example SDF file
OUT_SDF = Path(Path(POCKET_PDB).stem + "_mol.sdf")
# --- Model and device setup ---


model = load_model_using_config(CONFIG_YML, CKPT_PATH)

# --- Inference: Generate ligands ---
# If your model uses text conditioning, pass text_description. Otherwise, omit it.
with torch.no_grad():
    molecules = model.generate_ligands(
        pdb_file=POCKET_PDB,
        n_samples=N_SAMPLES,
        text_description=TEXT_DESCRIPTION,
        ref_ligand=LIGAND_SDF,
        sanitize=True,
        largest_frag=True,
        relax_iter=200,  # Optional: number of force field optimization steps
        n_nodes_min=12
    )

from rdkit import Chem


# # Save to SDF
w = Chem.SDWriter(str(OUT_SDF))
for mol in molecules:
    if mol is not None:
        w.write(mol)
w.close()

/tmp/ipykernel_1855108/3775149019.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ckpt_path, map_location='cpu')


Missing values: []
Unmatched values: []


[07:51:33] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
/home/sebastian/miniconda3/envs/diffsbdd_testrun/lib/python3.10/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
[07:51:55] Molecule does not have explicit Hs. Consider calling AddHs()
[07:51:55] Both bonds on one end of an atropisomer are on the same side - atoms are: 5 1
[07:51:55] The 2 defining bonds for an atropisomer are co-planar - atoms are: 4 1
[07:51:55] Both bonds on one end of an atropisomer are on the same side - atoms are: 4 5
[07:51:55] Molecule does not have explicit Hs. Consider calling AddHs()
[07:51:55] Both bonds on one end of an atropisomer are on the same side - atoms are: 8 4
[07:51:55] Molecule does not have explicit Hs. Consider calling AddHs()
[07:51:55] Molecule does no

In [5]:
model

LigandPocketDDPM(
  (ddpm): ConditionalDDPM(
    (gamma): PredefinedNoiseSchedule()
    (dynamics): EGNNDynamics(
      (atom_encoder): Sequential(
        (0): Linear(in_features=10, out_features=20, bias=True)
        (1): SiLU()
        (2): Linear(in_features=20, out_features=128, bias=True)
      )
      (atom_decoder): Sequential(
        (0): Linear(in_features=128, out_features=20, bias=True)
        (1): SiLU()
        (2): Linear(in_features=20, out_features=10, bias=True)
      )
      (residue_encoder): Sequential(
        (0): Linear(in_features=20, out_features=40, bias=True)
        (1): SiLU()
        (2): Linear(in_features=40, out_features=128, bias=True)
      )
      (residue_decoder): Sequential(
        (0): Linear(in_features=128, out_features=40, bias=True)
        (1): SiLU()
        (2): Linear(in_features=40, out_features=20, bias=True)
      )
      (egnn): EGNN(
        (embedding): Linear(in_features=129, out_features=256, bias=True)
        (embedding_out

In [8]:
import py3Dmol

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js',)
view.addModel(open(POCKET_PDB, 'r').read(), 'pdb')
view.setStyle({'model': -1}, {'cartoon': {'color': 'lime'}})
# view.addSurface(py3Dmol.VDW, {'opacity': 0.4, 'color': 'lime'})
view.addModelsAsFrames(open(OUT_SDF, 'r').read())
view.setStyle({'model': -1}, {'stick': {}})
view.zoomTo({'model': -1})
view.zoom(0.5)
# if target == "example (3rfm)":
#   view.rotate(90, 'y')
view.animate({'loop': "forward", 'interval': 1000})
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.